In [56]:
import paramiko, os
from getpass import getpass

# Create the passwd prompt
thePass = getpass(prompt="Please enter your SSH Password: ")


# Host Information
host = "192.168.6.71"
port = 2222
username = "paul.gleason"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

In [19]:
stdin, stdout, stderr = ssh.exec_command('pwd')
print(stdout.readlines())

['/home/paul.gleason\n']


In [20]:
sftp = ssh.open_sftp()

ssh.exec_command('cd /home/ubuntu/')

sftp.put('kraken', 'kraken') 

stdin, stdout, stderr = ssh.exec_command('ls')
print(stdout.readlines())


['kraken\n']


In [29]:
stdin, stdout, stderr = ssh.exec_command("/home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin")

if os.path.exists('kraken.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("kraken.txt")

for line in stdout.readlines():
    with open('kraken.txt', 'a') as f:
        f.write(line)

In [69]:
cmds = ["ls -al -h k /usr/bin/", "ls -al -h k /usr/sbin", "ls -al -h k /usr/local/bin", "ls -al -h k /usr/local/sbin"]

if os.path.exists('bin_results.txt'):
    usrinput = input("Would you like to delete this file? (Y/N) ")
    if usrinput == "Y" or usrinput == "y":
        os.remove("bin_results.txt")

for command in cmds:
    # Get the outpt form the command
    stdin, stdout, stderr = ssh.exec_command(command)

    # Get results from stdout
    lines = stdout.readlines()
    print(lines)

    # Cover the list to a string
    output = ''.join(lines)

    # Header output
    sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
    # Footer
    sepFooter = '' + '### END ' + command + ' ###\n\n'

    # Concatenate the header, output, and footer
    cmd_output = sepHeader + output + sepFooter

    # Sace the cmd_output to a file
    # a is append mode
    with open('bin_results.txt', 'a') as f:
        f.write(cmd_output)

['/usr/bin/:\n', 'total 120M\n', 'drwxr-xr-x  2 root   root     36K Oct 25 23:28 .\n', 'drwxr-xr-x 14 root   root    4.0K Feb  7  2022 ..\n', 'lrwxrwxrwx  1 root   root       4 Feb 17  2020 NF -> col1\n', '-rwxr-xr-x  1 root   root    139K Aug 16 09:23 VGAuthService\n', '-rwxr-xr-x  1 root   root     59K Sep  5  2019 [\n', '-rwxr-xr-x  1 root   root     31K May 19  2020 aa-enabled\n', '-rwxr-xr-x  1 root   root     35K May 19  2020 aa-exec\n', '-rwxr-xr-x  1 root   root    7.3K Oct 26  2021 add-apt-repository\n', '-rwxr-xr-x  1 root   root     31K Feb  7  2022 addpart\n', 'lrwxrwxrwx  1 root   root      26 Oct 20  2021 addr2line -> x86_64-linux-gnu-addr2line\n', '-rwxr-xr-x  1 root   root    2.5K Dec  4  2019 apport-bug\n', '-rwxr-xr-x  1 root   root     14K May 10 09:23 apport-cli\n', 'lrwxrwxrwx  1 root   root      10 May 10 09:23 apport-collect -> apport-bug\n', '-rwxr-xr-x  1 root   root    2.1K May 10 09:23 apport-unpack\n', 'lrwxrwxrwx  1 root   root       6 Feb 25  2020 apropos 

In [ ]:
thePass = getpass(prompt="Please enter your user Password: ")

stdin, stdout, stderr = ssh.exec_command("sudo -S lsof -i -n")
stdin.write(thePass + "\n")
stdin.flush()

for line in stdout.readlines():
    print(line)

In [55]:
ssh.close()

## Reflections

### Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?

### What did you like the most and least about this assignment?

### What additional questions do you have?
